# Fundamental Analysis using Dow30 Stocks

In [ ]:
import pandas as pd 
import numpy as np
import datetime
from datetime import date
import time
import pickle
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import xgboost as xgb
import sklearn as skl
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Bidirectional
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.metrics import RootMeanSquaredError, MeanSquaredError
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import os
import tensorflow as tf

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

In [346]:
dow30 = pd.read_csv('All_Data.csv')
dow30.rename(columns = {'date':'Date'}, inplace = True)
dow30 = dow30.drop(columns=['index'])
dow30['Date'] = pd.to_datetime(dow30['Date'])
dow30 = dow30.drop(columns=['high', 'low', 'open'])
dow30 = dow30.sort_values(by=['Stock', 'Date'], ascending=True)
dow30 = dow30.reset_index().drop(columns=['index'])

dates = dow30['Date']
stocks = dow30['Stock']
indices_to_dates = dates.to_dict()
indices_to_stocks = stocks.to_dict()
act_returns_mapping15 = dow30['15d_Return'].to_dict()
act_returns_mapping30 = dow30['30d_Return'].to_dict()
act_returns_mapping60 = dow30['60d_Return'].to_dict()
price_mapping = dow30['close'].to_dict()

dow30

## Feature Generation

### Percentage from monthly high, RSI

In [348]:
def get_monthly_highs(df):
    df = df.sort_values(by=['Stock', 'Date'], ascending=True)
    row = 0
    curr_month = df['Date'].loc[0].month
    highs=[]
    while (row < len(df)):
        closes = []
        count=0
        while (row<len(df)) and (df['Date'].loc[row].month == curr_month):
            closes.append(df['close'].loc[row])
            row+=1
            count+=1
        for i in range(0,count):
            highs.append(max(closes))
        if (row < len(df)):
            curr_month = df['Date'].loc[row].month
    
    return highs    
        

In [350]:
def get_rsi(prices):
    periods = 14
    ema = True
    close_delta = prices.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [352]:
highs = np.array(get_monthly_highs(dow30))
dow30['Percent_from_mth_high'] = (highs / dow30['close']) - 1

RSI = dow30.groupby('Stock').apply(lambda x: get_rsi(x['close']))
rsi = np.array(RSI)
dow30['RSI'] = rsi

dow30orig = dow30.copy()

dow30

,Date,10YBond,30dBondRet,VIX_high,spy_close,spy_5dret,spy_10dret,spy_15dret,surprise_%,expected_growth,previous_surprise,days_after_earn_report,dividends_change,prev_div_change,days_after_divid_report,close,volume,divCash,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y,Debt-to-Equity_Ratio,DividendsYield,PayoutRatio,Acc_Rec_Pay_Ration,Earnings_per_stock,15d_Return,30d_Return,60d_Return,Stock,Percent_from_mth_high,RSI
0,2019-03-11,2.643,-0.039956,16.43,260.370637,NaN,NaN,NaN,0.0096,-0.438095,0.0429,41.0,0.158730,0.105263,314.0,43.229318,128044136.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068977,0.159754,0.024331,AAPL,0.090497,NaN
1,2019-03-12,2.605,-0.050656,14.70,261.352498,NaN,NaN,NaN,0.0096,-0.438095,0.0429,42.0,0.158730,0.105263,315.0,43.715014,129870336.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072467,0.145100,0.027822,AAPL,0.078382,NaN
2,2019-03-13,2.611,-0.037242,14.05,263.082442,NaN,NaN,NaN,0.0096,-0.438095,0.0429,43.0,0.158730,0.105263,316.0,43.908325,124130096.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.075065,0.129712,0.050534,AAPL,0.073634,NaN
3,2019-03-14,2.630,-0.024119,13.84,262.914123,NaN,NaN,NaN,0.0096,-0.438095,0.0429,44.0,0.158730,0.105263,317.0,44.396437,94318032.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065096,0.111958,0.052262,AAPL,0.061830,NaN
4,2019-03-15,2.593,-0.015939,13.28,264.207467,NaN,NaN,NaN,0.0096,-0.438095,0.0429,45.0,0.158730,0.105263,318.0,44.973956,156171648.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058457,0.099345,0.050778,AAPL,0.048195,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

## Handle Missing Values

In [ ]:
na_df = dow30.set_index('Date')
na_df = na_df.isna()

In [ ]:
# columns with most NaN values
na_df.sum().sort_values(ascending=False).head(25)

In [ ]:
# dates with most NaN values
na_in_rows = dict()
for date, row in na_df.iterrows():
    na_in_rows[date] = row.sum()

In [ ]:
na_in_rows_df = pd.DataFrame(na_in_rows, index=[0]).stack()

In [ ]:
na_in_rows_df = pd.DataFrame(na_in_rows_df)

In [ ]:
na_in_rows_df.columns = ['#NaN values by date']

#### Dates before 2020-03-30 have large amounts of missing data due to fundamental data API only giving 3 years historical data, therefore remove dates before then

In [ ]:
na_in_rows_df.loc[na_in_rows_df['#NaN values by date'] != 0].tail(10)

In [355]:
dow30 = dow30.loc[dow30['Date'] > pd.to_datetime('2020-03-30')]

#### Find other na patterns 

In [ ]:
dow30.isna().sum()

#### Many missing dividends are due to Salesforce (CRM) not paying a dividend, therefore all of their dividend related features are NaN. Here, keeping these NaN is more appropriate than 0, as these features should not play into the model

In [ ]:
dow30[dow30['dividends_change'].isna()]

#### Look for source of numerous columns having 79 NaN's

In [ ]:
dow30[dow30['ppeq'].isna()]

#### Missing values in ppeq column correspond with missing values in most other fundamental metrics, making these NaN values large impact. These missing values are consecutive and occur in April 2020 for a few stocks (Covid?) Therefore, all of these rows can be removed safely.

In [354]:
dow30 = dow30.loc[~dow30['ppeq'].isna()]

#### Find source of 2284 missing values in '...Current' and '...NonCurrent' columns

In [ ]:
dow30[dow30['debtNonCurrent'].isna()].tail(50)

#### Missing values in these columns appear to be consecutive for specific quarters for a few stocks. Due to the small amount of features that are missing in each column, and additionally due to the expected low importance of the missing features, rows will remain in df unchanged.

#### Trailing PEG1Y appears to be missing for Q3 2021 for a single stock, data will be kept unchanged due to low impact

In [ ]:
dow30.loc[dow30['trailingPEG1Y'].isna()]

#### shareswaDil column is missing in a sizeable number of consecutive row blocks for various stocks (~1000), but impact is expected to be isolated as it is a single, low importance feature not intersecting with other NaN columns, therefore rows will be kept unchanged.

In [ ]:
dow30.loc[dow30['shareswaDil'].isna()]

In [ ]:
#dow30 = dow30.set_index('level_0')

In [356]:
dow30_final_with_date = dow30.copy() # Keep finalized version of dow30 df still with datestamps attached (for reference)
dow30_final_with_date

,Date,10YBond,30dBondRet,VIX_high,spy_close,spy_5dret,spy_10dret,spy_15dret,surprise_%,expected_growth,previous_surprise,days_after_earn_report,dividends_change,prev_div_change,days_after_divid_report,close,volume,divCash,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y,Debt-to-Equity_Ratio,DividendsYield,PayoutRatio,Acc_Rec_Pay_Ration,Earnings_per_stock,15d_Return,30d_Return,60d_Return,Stock,Percent_from_mth_high,RSI
267,2020-03-31,0.698,-0.551414,58.75,247.091952,0.060045,0.025844,-0.100848,0.0965,-0.584000,0.0704,63.0,0.054795,0.158730,336.0,62.256020,197002004.0,0.0,2.042100e+10,0.0,5.387700e+10,3.204000e+11,3.242100e+10,-2.789000e+09,3.334000e+09,2.419750e+11,3.067700e+10,0.000000e+00,3.588900e+10,5.928000e+09,4.017400e+10,1.766470e+11,0.000000e+00,1.526700e+11,7.842500e+10,3.318200e+10,0.0,1.437530e+11,9.879300e+10,1.095070e+11,8.908600e+10,1.458810e+11,9.609400e+10,1.750192e+10,1.135939,1.0,7.842500e+10,0.173146,1.495962,0.644858,0.383619,6.0,0.035599,0.005137,0.383619,3.331806,4.480937,9.013000e+09,-1.853000e+09,0.0,-2.094000e+10,1.697000e+09,1.384000e+09,-3.375000e+09,-176000000.0,8.030000e+08,-1.814600e+10,1.140700e+10,1.145800e+10,1.331100e+10,2.786000e+09,1.667800e+10,1.124900e+10,0.64,1.124900e+10,0.0,1.761876e+10,757000000.0,4.565000e+09,0.64,0.0,2.237000e+10,1.744040e+10,9.517000e+09,1.313500e+10,1.124900e+10,5.831300e+10,1.389200e+10,1.886000e+09,1.285300e+10,4.952000e+09,3.594300e+10,0.0,2.0,2020.0,1.112641e+12,1.181974e+12,19.446663,14.187323,5.462744,1.324104,-0.003033,-0.150872,0.946208,0.010280,0.085768,0.213038,0.438533,AAPL,0.190530,46.016459
268,2020-04-01,0.635,-0.595541,60.59,235.971616,-0.002593,0.031926,-0.097310,0.0965,-0.584000,0.0704,64.0,0.054795,0.158730,337.0,58.980289,176218552.0,0.0,2.042100e+10,0.0,5.387700e+10,3.204000e+11,3.242100e+10,-2.789000e+09,3.334000e+09,2.419750e+11,3.067700e+10,0.000000e+00,3.588900e+10,5.928000e+09,4.017400e+10,1.766470e+11,0.000000e+00,1.526700e+11,7.842500e+10,3.318200e+10,0.0,1.437530e+11,9.879300e+10,1.095070e+11,8.908600e+10,1.458810e+11,9.609400e+10,1.750192e+10,1.135939,1.0,7.842500e+10,0.173146,1.495962,0.644858,0.383619,6.0,0.035599,0.005137,0.383619,3.331806,4.480937,9.013000e+09,-1.853000e+09,0.0,-2.094000e+10,1.697000e+09,1.384000e+09,-3.375000e+09,-176000000.0,8.030000e+08,-1.814600e+10,1.140700e+10,1.145800e+10,1.331100e+10,2.786000e+09,1.667800e+10,1.124900e+10,0.64,1.124900e+10,0.0,1.761876e+10,757000000.0,4.565000e+09,0.64,0.0,2.237000e+10,1.744040e+10,9.517000e+09,1.313500e+10,1.124900e+10,5.831300e+10,1.389200e+10,1.886000e+09,1.285300e+10,4.952000e+09,3.594300e+10,0.0,2.0,2020.0,1.054097e+12,1.123430e+12,18.423436,13.440827,5.175311,1.324104,-0.003202,-0.150872,0.946208,0.010851,0.141630,0.288275,0.471774,AAPL,0.219543,41.996987
269,2020-04-02,0.627,-0.588852,57.24,241.416746,-0.035873,0.053499,0.021229,0.0965,-0.584000,0.0704,65.0,0.054795,0.158730,338.0,59.964478,165933972.0,0.0,2.042100e+10,0.0,5.387700e+10,3.204000e+11,3.242100e+10,-2.789000e+09,3.334000e+09,2.419750e+11,3.067700e+10,0.000000e+00,3.588900e+10,5.928000e+09,4.017400e+10,1.766470e+11,0.000000e+00,1.526700e+11,7.842500e+10,3.318200e+10,0.0,1.437530e+11,9.879300e+10,1.095070e+11,8.90

In [23]:
#dow30 = dow30.drop(columns=['spy_5dret', 'spy_10dret', 'spy_15dret', 'spy_close'])

In [59]:
# execute this cell if dow30 needs to have date reinserted
#dow30 = dow30_final_with_date

#### Keep one sample per business week (1 entry for every 5). Remaining 4/5 will be kept as a final test set. Due to the scope of the model, the minute changes in the features on a day-to-day basis, and the intention for the predictive power to be directed longer term (15d, 30d, 60d, +), keeping all the daily data is unnecessary and will likely cause overfitting quickly

In [357]:
true_test = dow30.loc[dow30.index % 5 != 0] # large test set
train = dow30.loc[dow30.index % 5 == 0] # smaller train set (yes train set is smaller)
true_test = true_test.loc[true_test['Date'] < pd.to_datetime('2023-01-30')]

#### Remove zeroes in target to prevent infinite error when using mape

In [358]:
train = train.loc[~(train['15d_Return'] == 0)]
train = train.loc[~(train['30d_Return'] == 0)]
train = train.loc[~(train['60d_Return'] == 0)]
true_test = true_test.loc[~(true_test['15d_Return'] == 0)]
true_test = true_test.loc[~(true_test['30d_Return'] == 0)]
true_test = true_test.loc[~(true_test['60d_Return'] == 0)]

#### Split into train and test sets

In [26]:
x_trn = train.drop(columns=['15d_Return', '30d_Return', '60d_Return'])
x_trn = x_trn.loc[x_trn['Date'] < pd.to_datetime('2023-01-30')]
x_trn = x_trn.drop(columns=['Date'])

y_trn15 = train[['15d_Return','Date']]
y_trn15 = y_trn15.loc[y_trn15['Date'] < pd.to_datetime('2023-01-30')]
y_trn15 = y_trn15.drop(columns=['Date'])

y_trn30 = train[['30d_Return', 'Date']]
y_trn30 = y_trn30.loc[y_trn30['Date'] < pd.to_datetime('2023-01-06')]
y_trn30 = y_trn30.drop(columns=['Date'])

y_trn60 = train[['60d_Return', 'Date']]
y_trn60 = y_trn60.loc[y_trn60['Date'] < pd.to_datetime('2022-11-22')]
y_trn60 = y_trn60.drop(columns=['Date'])

# Must remove date and stock (cat. type) columns before making DMatrix
#dtrain15 = xgb.DMatrix(x_trn, y_trn15)

# 15 Day Return Prediction

In [ ]:
reg = XGBRegressor()

#### Perform Grid Search to Optimize Parameters

In [ ]:
# first iteration of grid search params
params = {
    'max_depth'        : range(5, 15, 5),
    'min_child_weight' : range(9, 21, 3),
    'gamma'            : [0.0, 0.5, 1, 10],
    'colsample_bytree' : [0.3, 0.5, 0.7],
    'n_estimators'     : range(10, 70, 20)
    
}
best1 = {
    'colsample_bytree': 0.7,
    'gamma': 0.5,
    'max_depth': 5,
    'min_child_weight': 9,
    'n_estimators': 50
}
bestscore1 = 1.677
# optimal params from first grid search with a range to either side for secondary grid search
params1 = {
    'colsample_bytree' : [0.6, 0.7, 0.8],
    'gamma'            : [0.3, 0.5, 0.7],
    'max_depth'        : range(4, 6, 1),
    'min_child_weight' : range(7, 11, 1),
    'n_estimators'     : range(40, 60, 5)  

}
best2 = {
    'colsample_bytree': 0.8,
    'gamma': 0.5,
    'max_depth': 4,
    'min_child_weight': 7,
    'n_estimators': 45,
}
bestscore2 = 1.6765

In [ ]:
gridp = GridSearchCV(reg, param_grid=params1, n_jobs=-1, scoring='neg_mean_squared_error', cv=5, verbose=3)

In [ ]:
gridp.fit(x_trn, y_trn15)

#### Evaluate results of Random Search

In [ ]:
print(gridp.best_estimator_)

In [ ]:
pd.DataFrame(gridp.cv_results_).sort_values(by=['rank_test_score'], ascending = True)

In [ ]:
best_first = gridp.best_params_  

In [ ]:
opt_paramsp = gridp.best_params_
opt_paramsp

#### Train model using optimal parameters

In [ ]:
model_cv15 = xgb.cv(opt_paramsp, dtrain15, nfold=5, num_boost_round = 50, metrics='mse', as_pandas=True, early_stopping_rounds=1, shuffle=True)

In [ ]:
model_cv15

In [ ]:
trn_15 = xgb.DMatrix(x_trn, y_trn15)

In [ ]:
tsn_15 = xgb.DMatrix(true_test.drop(columns=['15d_Return', '30d_Return', '60d_Return','Date']), true_test['15d_Return'])

In [ ]:
trainer15 = xgb.train(best2, dtrain15, num_boost_round=30, evals=[(trn_15, 'trnp'), (tsn_15, 'tsnp')], early_stopping_rounds=1)

In [ ]:
with open('model_15.pickle', 'wb') as file:
    pickle.dump(reg15, file)

In [ ]:
with open('model_15.pickle', 'rb') as file:
    trainer15 = pickle.load(file)

In [ ]:
most_recent_test15 = xgb.DMatrix(February_df.drop(columns=['15d_Return', '30d_Return', '60d_Return', 'Date']))

## Train model using XGBRegressor object (easier implementation)

In [306]:
reg15 = XGBRegressor(colsample_bytree=1, gamma=0.0, max_depth=10, min_child_weight=2, 
                     n_estimators=45, early_stopping_rounds=1)

#### Retrain model excluding Jan 2023 from train set, to have proper comparison

In [307]:
x_trn = train.loc[train['Date'] < pd.to_datetime('2022-01-01')]
y_trn15 = x_trn.copy()
y_trn15 = y_trn15['15d_Return']
x_trn = x_trn.drop(columns=['Date','Stock','15d_Return','30d_Return', '60d_Return'])
x_tst = dow30.loc[dow30['Date'] > pd.to_datetime('2022-01-01')]
x_tst = x_tst.loc[x_tst['Date'] < pd.to_datetime('2023-01-01')]
y_tst15 = x_tst.copy() 
x_tst = x_tst.drop(columns=['Date','15d_Return', '30d_Return', '60d_Return', 'Stock'])
y_tst15 = y_tst15['15d_Return']

In [320]:
reg15.fit(x_trn, y_trn15, eval_set=[(x_tst, y_tst15)])

[0]	validation_0-rmse:0.36538
[1]	validation_0-rmse:0.26777
[2]	validation_0-rmse:0.20285
[3]	validation_0-rmse:0.16477
[4]	validation_0-rmse:0.13774
[5]	validation_0-rmse:0.12042
[6]	validation_0-rmse:0.10746
[7]	validation_0-rmse:0.10067
[8]	validation_0-rmse:0.09558
[9]	validation_0-rmse:0.09197
[10]	validation_0-rmse:0.08959
[11]	validation_0-rmse:0.08765
[12]	validation_0-rmse:0.08624
[13]	validation_0-rmse:0.08586
[14]	validation_0-rmse:0.08551
[15]	validation_0-rmse:0.08502
[16]	validation_0-rmse:0.08480
[17]	validation_0-rmse:0.08454
[18]	validation_0-rmse:0.08436
[19]	validation_0-rmse:0.08432
[20]	validation_0-rmse:0.08429
[21]	validation_0-rmse:0.08412
[22]	validation_0-rmse:0.08404
[23]	validation_0-rmse:0.08371
[24]	validation_0-rmse:0.08377


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=1, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=45, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [429]:
preds = reg15.predict(dow30.loc[dow30['Date'] > pd.to_datetime('2022-01-01')].drop(columns=['15d_Return', '30d_Return', '60d_Return','Date', 'Stock']))

In [166]:
#preds = pd.DataFrame(trainer15.predict(most_recent_test15))

In [326]:
output15 = dow30.copy()
output15 = output15[['15d_Return','Date']]

output15 = output15.loc[output15['Date'] > pd.to_datetime('2022-01-01')]
output15['Predicted 15d_Return'] = preds

output15['Date'] = output15.index.map(indices_to_dates)
output15['Stock'] = output15.index.map(indices_to_stocks)

output15['Date'] = pd.to_datetime(output15['Date'])
JanFeb_results15 = output15.loc[output15['Date'] > pd.to_datetime('2022-01-01')]

JanFeb_results15 = JanFeb_results15.sort_values(by=['Predicted 15d_Return', 'Date'], ascending=True)

In [434]:
Jan_results15 = JanFeb_results15.loc[~JanFeb_results15['15d_Return'].isna()]

In [ ]:
JanFeb_results15

In [ ]:
JanFeb_results15[JanFeb_results15['Date'] > pd.to_datetime('2023-02-14')]

In [435]:
print(mean_absolute_error(Jan_results15['Predicted 15d_Return'], Jan_results15['15d_Return']))

0.06019836764376099


In [ ]:
# January results from true_test df (model included Jan) achieved 0.03228 mae (+/- 3.2% return on avg)
# January results from train df (model excluded Jan) achieved 0.0521 mae (+/- 5.2% return on avg)
# January results from train df (excluded Jan) achieved 0.04976 mae (after addition of rsi and %from mthly high)

In [ ]:
Jan_results15.sort_values(by=['Predicted 15d_Return'])

# Profit Analysis

Model will be assumed to be executed every 2 weeks, where the top n predicted returns on the day of model execution will be purchased. Then, two weeks later, the positions will be exited, and positions for the n highest predicted returns on that day will be entered. Profit will be averaged by switching the day of model execution (Every other monday->Every other tuesday..etc). An overall avg profit will then be developed using this specific execution strategy. Further optimization work or even a future ML model can tackle the optimal number of n stocks to divide the allocation amongst as well as any staggerred position entry/exit.

In [437]:
profit_analysis15 = Jan_results15.sort_values('Date')

profit_analysis15

,15d_Return,Date,Predicted 15d_Return,Stock
711,-0.122136,2022-01-03,-0.040589,AAPL
1712,-0.007279,2022-01-03,0.043935,AMGN
24735,0.082504,2022-01-03,0.010904,TRV
27738,0.020675,2022-01-03,0.000810,VZ
23734,-0.015368,2022-01-03,0.040646,PG
...,...,...,...,...
25008,0.014062,2023-02-03,0.040656,TRV
26009,0.023940,2023-02-03,0.018023,UNH
27010,-0.040619,2023-02-03,-0.040407,V
13997,-0.035346,2023-02-03,-0.022738,IBM


Add 1-14d returns as columns, to aid in prediction and loss-stopping. A future model may be trained on this dataset to suggest changes to positions after entry.

In [438]:
for i in range(1, 11):
    profit_analysis15['%sd_Return'%(i)] = profit_analysis15.index + i
    profit_analysis15['%sd_Return'%(i)] = (profit_analysis15['%sd_Return'%(i)].map(price_mapping) - profit_analysis15.index.map(price_mapping)) /  profit_analysis15.index.map(price_mapping)

    profit_analysis15

# divide profit_analysis15 into results for each day of the business week
pa15mon = profit_analysis15.loc[profit_analysis15['Date'].dt.dayofweek == 0]
pa15tues = profit_analysis15.loc[profit_analysis15['Date'].dt.dayofweek == 1]
pa15wed = profit_analysis15.loc[profit_analysis15['Date'].dt.dayofweek == 2]
pa15th = profit_analysis15.loc[profit_analysis15['Date'].dt.dayofweek == 3]
pa15fri = profit_analysis15.loc[profit_analysis15['Date'].dt.dayofweek == 4]

pa15mon = pa15mon.sort_values(by=['Date', 'Predicted 15d_Return'], ascending=True)
pa15tues = pa15tues.sort_values(by=['Date', 'Predicted 15d_Return'], ascending=True)
pa15wed = pa15wed.sort_values(by=['Date', 'Predicted 15d_Return'], ascending=True)
pa15th = pa15th.sort_values(by=['Date', 'Predicted 15d_Return'], ascending=True)
pa15fri = pa15fri.sort_values(by=['Date', 'Predicted 15d_Return'], ascending=True)

In [439]:
profits15 = pd.DataFrame(columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], index=range(1,10))

# loop over number of stocks to split positions in each entry, and see which yields best profit
for i in range(1, 10):
    mon = pd.DataFrame(pa15mon.groupby('Date')['Predicted 15d_Return'].nlargest(i)).reset_index().set_index('level_1')
    mon['Stock'] = mon.index.map(indices_to_stocks)
    mon['15d_Return'] = mon.index.map(act_returns_mapping15)
    weeks_mon = mon.groupby('Date')['15d_Return'].mean()
    profits15['Monday'].loc[i] = weeks_mon.mean()
    
    tues = pd.DataFrame(pa15tues.groupby('Date')['Predicted 15d_Return'].nlargest(i)).reset_index().set_index('level_1')
    tues['Stock'] = tues.index.map(indices_to_stocks)
    tues['15d_Return'] = tues.index.map(act_returns_mapping15)
    weeks_tues = tues.groupby('Date')['15d_Return'].mean()
    profits15['Tuesday'].loc[i] = weeks_tues.mean()

    wed = pd.DataFrame(pa15wed.groupby('Date')['Predicted 15d_Return'].nlargest(i)).reset_index().set_index('level_1')
    wed['Stock'] = wed.index.map(indices_to_stocks)
    wed['15d_Return'] = wed.index.map(act_returns_mapping15)
    weeks_wed = wed.groupby('Date')['15d_Return'].mean()
    profits15['Wednesday'].loc[i] = weeks_wed.mean()

    th = pd.DataFrame(pa15th.groupby('Date')['Predicted 15d_Return'].nlargest(i)).reset_index().set_index('level_1')
    th['Stock'] = th.index.map(indices_to_stocks)
    th['15d_Return'] = th.index.map(act_returns_mapping15)
    weeks_th = th.groupby('Date')['15d_Return'].mean()
    profits15['Thursday'].loc[i] = weeks_th.mean()

    fri = pd.DataFrame(pa15fri.groupby('Date')['Predicted 15d_Return'].nlargest(i)).reset_index().set_index('level_1')
    fri['Stock'] = fri.index.map(indices_to_stocks)
    fri['15d_Return'] = fri.index.map(act_returns_mapping15)
    weeks_fri = fri.groupby('Date')['15d_Return'].mean()
    profits15['Friday'].loc[i] = weeks_fri.mean()

In [440]:
profits15 # shows mean return for entering positions in stocks with the n highest predicted returns on the specified weekday
# over a 15d holding period

,Monday,Tuesday,Wednesday,Thursday,Friday
1,0.03658,0.039558,0.046356,0.047609,0.021667
2,0.032667,0.037756,0.032586,0.03263,0.020711
3,0.029626,0.033507,0.031702,0.03051,0.022964
4,0.025685,0.027601,0.02934,0.028662,0.022034
5,0.022656,0.024148,0.028336,0.027062,0.023074
6,0.022888,0.022841,0.02535,0.025396,0.022418
7,0.020197,0.02412,0.022773,0.023017,0.01995
8,0.019264,0.024011,0.020751,0.020681,0.019246
9,0.017865,0.023159,0.019913,0.018714,0.018177


In [441]:
# maximum profit: 2 stocks on every third tuesday
tues = pd.DataFrame(pa15tues.groupby('Date')['Predicted 15d_Return'].nlargest(1)).reset_index()
tues['Stock'] = tues['level_1'].map(indices_to_stocks)
tues['15d_Return'] = tues['level_1'].map(act_returns_mapping15)
tues1 = tues.loc[(tues.index % 6 == 2) | (tues.index % 6 == 3)] # Takes the two stocks every third tuesday

In [442]:
for i in range(1, 11):
    tues['%sd_Return'%(i)] = tues['level_1'] + i
    tues['%sd_Return'%(i)] = (tues['%sd_Return'%(i)].map(price_mapping) - tues['level_1'].map(price_mapping)) /  tues['level_1'].map(price_mapping)


In [215]:
tues[tues['Date'].dt.month==9]

,Date,level_1,Predicted 15d_Return,Stock,15d_Return,1d_Return,2d_Return,3d_Return,4d_Return,5d_Return,6d_Return,7d_Return,8d_Return,9d_Return,10d_Return
35,2022-09-06,19880,0.108696,MMM,-0.035935,0.033877,0.022899,0.055746,0.065609,0.033190,0.007976,-0.001544,0.000086,0.000343,-0.000686
36,2022-09-13,19885,0.107198,MMM,-0.040259,-0.024404,-0.033618,-0.032041,-0.031792,-0.032788,-0.046983,-0.052544,-0.062090,-0.062007,-0.066905
37,2022-09-20,19890,0.162543,MMM,-0.065911,-0.014676,-0.020426,-0.030295,-0.030209,-0.035273,-0.019224,-0.036217,-0.051665,-0.028321,-0.007724
38,2022-09-27,27895,0.167077,VZ,-0.030362,0.012857,-0.006428,-0.023656,0.006943,0.023656,0.013114,-0.010221,-0.036117,-0.042133,-0.059919


In [443]:
mae = tues1.groupby('Date')['15d_Return'].mean()
funds = 1
for week in mae:
    funds *= (1+week)
funds

2.0733962846955087

#### 1.702999x initial investment starting first tuesday of the year, 1.285x for second tuesday, 2.722x starting third tuesday. To balance these out, a 2-stock purchase every tuesday, with a 1/6 allocation to each position is suggested

### With 2-stock buy/sell each tuesday with 1/3 allocation on each day, we achieve a 1.925x initial investment return over a year

In [184]:
tues_cleaned = tues # run for no interference trading, run line below for avoiding trading 
#tues_cleaned = tues.loc[(tues['Date'] < pd.to_datetime('2022-08-15')) | (tues['Date'] > pd.to_datetime('2022-09-25'))]
funds = 0
alloc = 1.0/6.0
all1, all2, all3, all4, all5, all6 = alloc, alloc, alloc, alloc, alloc, alloc
for num, ret in enumerate(tues_cleaned['15d_Return']):
    n = num % 6
    if n == 0:
        all1 *= (1+ret)
    elif n == 1:
        all2 *= (1+ret)
    elif n == 2:
        all3 *= (1+ret)
    elif n == 3:
        all4 *= (1+ret)
    elif n == 4:
        all5 *= (1+ret)
    else:
        all6 *= (1+ret)
funds = all1 + all2 + all3 + all4 + all5 + all6
print('If early exit is not allowed, Multiplier at the end of the year is: %sx'%(funds))

If early exit is not allowed, Multiplier at the end of the year is: 1.6545717512111993x


In [444]:
# for nlargest(1)
tues_cleaned = tues # run for no interference trading, run line below for avoiding trading 
#tues_cleaned = tues.loc[(tues['Date'] < pd.to_datetime('2022-08-15')) | (tues['Date'] > pd.to_datetime('2022-09-25'))]
funds = 0
alloc = 1.0/3.0
all1, all2, all3= alloc, alloc, alloc
for num, ret in enumerate(tues_cleaned['15d_Return']):
    n = num % 3
    if n == 0:
        all1 *= (1+ret)
    elif n == 1:
        all2 *= (1+ret)
    else:
        all3 *= (1+ret)
funds = all1 + all2 + all3 
print('If early exit is not allowed, Multiplier at the end of the year is: %sx'%(funds))

If early exit is not allowed, Multiplier at the end of the year is: 2.0344904212269506x


In [457]:
tues[tues['15d_Return'] < 0]['15d_Return'].mean()

-0.04102481328105927

### Try a stop loss or gain trigger rule (if x% of predicted return is realized on day n, exit position on day n and do not reinvest those funds until the full 15day period is over, regardless of n

In [89]:
funds = 0
alloc = 1.0/6.0
all1, all2, all3, all4, all5, all6 = alloc, alloc, alloc, alloc, alloc, alloc
thresh = 2.5
for i, row in tues.iterrows():
    ret = row['15d_Return']
    for n in range(1, 10):
        if (row['%sd_Return'%(n)] > (thresh * row['Predicted 15d_Return'])):
            ret = row['%sd_Return'%(n)]
            break
    j = i % 6
    if j == 0:
        all1 *= (1+ret)
    elif j == 1:
        all2 *= (1+ret)
    elif j == 2:
        all3 *= (1+ret)
    elif j == 3:
        all4 *= (1+ret)
    elif j == 4:
        all5 *= (1+ret)
    else:
        all6 *= (1+ret)      
funds = all1 + all2 + all3 + all4 + all5 + all6
print(f'Multiplier at the end of the year using {thresh}x return threshold for early exit:{funds}')

KeyError: '1d_Return'

#### Percentage Accuracy: measured as percentage of top 2 predicted returns that yield a return > r%

In [218]:
for r in range(0, 9):
    perc = len(tues.loc[tues['15d_Return'] > (r * 0.01)]) / len(tues)
    print(f'percentage of actual returns greater than{r}% is:{perc*100}%')

percentage of actual returns greater than0% is:64.91228070175438%
percentage of actual returns greater than1% is:61.40350877192983%
percentage of actual returns greater than2% is:57.89473684210527%
percentage of actual returns greater than3% is:56.14035087719298%
percentage of actual returns greater than4% is:54.385964912280706%
percentage of actual returns greater than5% is:49.122807017543856%
percentage of actual returns greater than6% is:47.368421052631575%
percentage of actual returns greater than7% is:40.35087719298245%
percentage of actual returns greater than8% is:36.84210526315789%


#### Percentage Accuracy for nlargest = (choice) for all days

In [219]:
choice = 1
week = pd.DataFrame(profit_analysis15.groupby('Date')['Predicted 15d_Return'].nlargest(choice)).reset_index()
week['Stock'] = week['level_1'].map(indices_to_stocks)
week['15d_Return'] = week['level_1'].map(act_returns_mapping15)
for i in range(1, 11):
    week['%sd_Return'%(i)] = week['level_1'] + i
    week['%sd_Return'%(i)] = (week['%sd_Return'%(i)].map(price_mapping) - week['level_1'].map(price_mapping)) /  week['level_1'].map(price_mapping)


In [220]:
for r in range(0, 15):
    perc = len(week.loc[week['15d_Return'] > (r * 0.01)]) / len(week)
    print(f'percentage of actual returns greater than {r}% is:{perc*100}%')

percentage of actual returns greater than 0% is:64.23357664233576%
percentage of actual returns greater than 1% is:59.12408759124088%
percentage of actual returns greater than 2% is:56.56934306569343%
percentage of actual returns greater than 3% is:53.284671532846716%
percentage of actual returns greater than 4% is:49.63503649635037%
percentage of actual returns greater than 5% is:46.715328467153284%
percentage of actual returns greater than 6% is:43.06569343065693%
percentage of actual returns greater than 7% is:37.591240875912405%
percentage of actual returns greater than 8% is:33.21167883211679%
percentage of actual returns greater than 9% is:30.29197080291971%
percentage of actual returns greater than 10% is:25.91240875912409%
percentage of actual returns greater than 11% is:23.722627737226276%
percentage of actual returns greater than 12% is:21.16788321167883%
percentage of actual returns greater than 13% is:18.248175182481752%
percentage of actual returns greater than 14% is:15.6

### Percentage Accuracy for each month after train-test boundary (does accuracy decrease over time)

#### results show little to no discernable correlation with time after train-test boundary, but indicates a huge reliance on market factors- accuracy for correct >0% return prediction as low as 23% in Sept2022, and as high as 95% for Oct2022

In [221]:
week2 = week[week['Date'].dt.year == 2022]
for mth in range(1, 13):
    mthloop = week2[week2['Date'].dt.month == mth]
    for r in range(0, 10):
        perc = len(mthloop.loc[mthloop['15d_Return'] > ((r * 0.1)*mthloop['Predicted 15d_Return'])]) / len(mthloop)
        print(f'percentage of actual returns greater than {r*10}% of prediction in month {mth}is:{perc*100}%')
    print('----------------------------------------------------------')

percentage of actual returns greater than 0% of prediction in month 1is:75.0%
percentage of actual returns greater than 10% of prediction in month 1is:70.0%
percentage of actual returns greater than 20% of prediction in month 1is:65.0%
percentage of actual returns greater than 30% of prediction in month 1is:65.0%
percentage of actual returns greater than 40% of prediction in month 1is:45.0%
percentage of actual returns greater than 50% of prediction in month 1is:25.0%
percentage of actual returns greater than 60% of prediction in month 1is:25.0%
percentage of actual returns greater than 70% of prediction in month 1is:25.0%
percentage of actual returns greater than 80% of prediction in month 1is:20.0%
percentage of actual returns greater than 90% of prediction in month 1is:15.0%
----------------------------------------------------------
percentage of actual returns greater than 0% of prediction in month 2is:57.89473684210527%
percentage of actual returns greater than 10% of prediction i

In [191]:
tues

,Date,level_1,Predicted 15d_Return,Stock,15d_Return,1d_Return,2d_Return,3d_Return,4d_Return,5d_Return,6d_Return,7d_Return,8d_Return,9d_Return,10d_Return
0,2022-01-04,7712,0.096276,CVX,0.092728,0.006506,0.015070,0.029647,0.030306,0.053858,0.048588,0.044223,0.062011,0.065470,0.061599
1,2022-01-11,24717,0.044199,TRV,0.058593,-0.001165,-0.000674,0.000306,0.002513,-0.018938,0.012380,-0.003371,0.034751,0.033342,0.028806
2,2022-01-18,10721,0.123016,GS,0.044300,-0.019977,-0.017777,-0.029599,-0.031067,-0.036258,-0.033070,-0.037726,-0.020852,0.000790,0.027257
3,2022-01-25,10726,0.156443,GS,0.065554,0.003308,-0.001522,0.015986,0.038442,0.065905,0.062978,0.050739,0.076270,0.077880,0.083590
4,2022-02-01,7731,0.057401,CVX,0.016275,0.004600,-0.004377,0.008087,0.027895,0.012241,0.022257,0.009274,0.029824,0.013948,0.006603
5,2022-02-08,20736,0.090878,MRK,-0.003121,-0.004941,-0.004551,-0.003511,-0.005721,0.011702,0.004031,-0.007151,-0.007021,-0.012482,-0.013912
6,2022-02-15,10741,0.059556,GS,-0.077807,-0.010689,-0.040886,-0.049184,-0.054047,-0.062510,-0.065258,-0.037973,-0.062236,-0.092708,-0.070094
7,2022-02-22,11745,0.112673,HD,0.045349,-0.024417,-0.009267,0.001518,-0.001075,0.012904,0.035424,0.025904,0.025588,0.020875,0.001803
8,2022-03-01,7750,0.117009,CVX,0.095979,0.029522,0.043414,0.059645,0.082287,0.138993,0.110540,0.140930,0.141464,0.113545,0.057173
9,2022-03-08,2755,0.130827,AXP,0.193002,0.053733,0.059669,0.049047,0.079600,0.099906,0.128460,0.168197,0.191628,0.181443,0.195439


# 15d Run for use

In [423]:
reg15 = XGBRegressor(colsample_bytree=0.9, gamma=0.0, max_depth=10, min_child_weight=1, 
                     n_estimators=45, early_stopping_rounds=1)

### Forward fill last day or two of fundamental data if needed

In [361]:
# make predictions df, with any NaN values forward filled from previous day in the vicinity
dow30_lastdays = dow30orig.loc[dow30orig['Date'] > pd.to_datetime('2023-02-26')] 
dow30_lastdays.ffill(axis=0, inplace=True)

x_trn = train.loc[train['Date'] < pd.to_datetime('2022-01-01')]
y_trn15 = x_trn.copy()
y_trn15 = y_trn15['15d_Return']
x_trn = x_trn.drop(columns=['Date','Stock','15d_Return','30d_Return', '60d_Return'])
x_tst = dow30.loc[dow30['Date'] > pd.to_datetime('2022-01-01')]
x_tst = x_tst.loc[x_tst['Date'] < pd.to_datetime('2023-01-01')]
y_tst15 = x_tst.copy() 
x_tst = x_tst.drop(columns=['Date','15d_Return', '30d_Return', '60d_Return', 'Stock'])
y_tst15 = y_tst15['15d_Return']

dow30_lastdays

In [447]:
reg15.fit(x_trn, y_trn15, eval_set=[(x_tst, y_tst15)])

[0]	validation_0-rmse:0.36554
[1]	validation_0-rmse:0.26768
[2]	validation_0-rmse:0.20265
[3]	validation_0-rmse:0.16059
[4]	validation_0-rmse:0.13363
[5]	validation_0-rmse:0.11610
[6]	validation_0-rmse:0.10268
[7]	validation_0-rmse:0.09496
[8]	validation_0-rmse:0.09029
[9]	validation_0-rmse:0.08658
[10]	validation_0-rmse:0.08402
[11]	validation_0-rmse:0.08263
[12]	validation_0-rmse:0.08168
[13]	validation_0-rmse:0.08099
[14]	validation_0-rmse:0.08036
[15]	validation_0-rmse:0.07981
[16]	validation_0-rmse:0.07963


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=1,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=45, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [445]:
with open('reg_15.pickle', 'wb') as file:
    pickle.dump(reg15, file)

In [448]:
with open('reg_15.pickle', 'rb') as file:
    reg15 = pickle.load(file)

In [450]:
preds = reg15.predict(dow30_lastdays.drop(columns=['15d_Return', '30d_Return', '60d_Return','Date', 'Stock']))

In [166]:
#preds = pd.DataFrame(trainer15.predict(most_recent_test15))

In [452]:
Predictions = dow30_lastdays.copy()
Predictions = Predictions[['15d_Return','Date']]
Predictions['Predicted 15d_Return'] = preds
Predictions['Date'] = Predictions.index.map(indices_to_dates)
Predictions['Stock'] = Predictions.index.map(indices_to_stocks)
Predictions['Date'] = pd.to_datetime(Predictions['Date'])
Results = Predictions.sort_values(by=['Predicted 15d_Return', 'Date'], ascending=False)
Results

,15d_Return,Date,Predicted 15d_Return,Stock
28026,NaN,2023-02-27,0.134321,VZ
9007,NaN,2023-02-27,0.128076,DIS
15013,NaN,2023-02-27,0.115284,INTC
28027,NaN,2023-02-28,0.113328,VZ
15014,NaN,2023-02-28,0.105826,INTC
12010,NaN,2023-02-27,0.104545,HD
12011,NaN,2023-02-28,0.103801,HD
8007,NaN,2023-02-28,0.094964,CVX
9008,NaN,2023-02-28,0.094359,DIS
22021,NaN,2023-02-28,0.090800,MSFT


## Neural Network Alternative to XGBoost

In [267]:
sc = StandardScaler()
x_trnfs = sc.fit_transform(x_trn)
y_trn15fs = sc.fit_transform(np.array(y_trn15).reshape(-1,1))
x_tstfs = sc.fit_transform(x_tst)
y_tst15fs = sc.fit_transform(np.array(y_tst15).reshape(-1,1))

In [285]:
model=Sequential()
model.add(Dense(106, input_shape=(106,1)))
model.add(Dense(72))
model.add(Dense(36))
model.add(Dense(20))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
#model.add(Conv1D(filters=24, kernel_size=1, padding='same', activation='tanh',input_shape=(1, 6)))
#model.add(MaxPooling1D(pool_size=1, padding='same'))
#model.add(LSTM(24, activation='tanh'))

model.fit(x=x_trnfs, y=y_trn15fs, validation_data=(x_tstfs,y_tst15fs), epochs=100,verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
name = '15d'+'_model.pickle'
with open(name, 'wb') as file:
    pickle.dump(model, file)

Epoch 1/100
21/21 [==============================] - 3s 101ms/step - loss: nan - val_loss: nan
Epoch 2/100
21/21 [==============================] - 2s 104ms/step - loss: nan - val_loss: nan
INFO:tensorflow:Assets written to: ram://0975f93e-989f-440f-a053-15e8bd0ee268/assets


In [280]:
pd.DataFrame(y_tst15fs).isna().sum()

0    0
dtype: int64

In [287]:
x_trn.isna().sum().sum()

3649

In [290]:
pd.DataFrame(x_trnfs).isna().sum().sum()

3649

## Profit Analysis based on Stock

In [ ]:
by_stock = profit_analysis15.groupby('Stock')['Predicted 15d_Return', '15d_Return'].mean()

In [ ]:
by_stock['Mean_abs_error'] = abs(by_stock['Predicted 15d_Return'] - by_stock['15d_Return'])

In [ ]:
by_stock.sort_values(by=['Mean_abs_error'], ascending=True)

#### Now compute mae for predictions above a 5% Predicted Return threshold

In [ ]:
by_stock_top = profit_analysis15.loc[profit_analysis15['Predicted 15d_Return'] > 0.05].groupby('Stock')['Predicted 15d_Return', '15d_Return'].mean()

In [ ]:
by_stock_top['Mean_abs_error'] = abs(by_stock_top['Predicted 15d_Return'] - by_stock_top['15d_Return'])

In [ ]:
by_stock_top.sort_values(by=['Mean_abs_error'], ascending=True)

### Feature importances

In [ ]:
importances15 = pd.DataFrame(reg15.get_booster().get_score(importance_type='gain'), index=[0]).transpose()
importances15.columns = ['gain']
#importances15 = pd.DataFrame(reg15.get_booster().get_score(importance_type='weight'), index=[0]).transpose()
#importances15.columns = ['weight']

In [ ]:
importances15.sort_values(by=['gain'], ascending = False)
#importances15.sort_values(by=['weight'], ascending = False)

# 30 Day Return Prediction

#### Retrain model excluding Jan 2023 from train set, to have proper comparison

In [ ]:
reg30 = XGBRegressor(colsample_bytree=1, gamma=0.0, max_depth=10, min_child_weight=2, 
                     n_estimators=45, early_stopping_rounds=1)

In [ ]:
x_trn = train.loc[train['Date'] < pd.to_datetime('2022-01-01')]
y_trn30 = x_trn.copy()
y_trn30 = y_trn30['30d_Return']
x_trn = x_trn.drop(columns=['Date','Stock','15d_Return','30d_Return', '60d_Return'])

In [ ]:
reg30.fit(x_trn, y_trn30, eval_set=[(x_trn, y_trn30)])

In [ ]:
preds30 = reg30.predict(dow30.loc[dow30['Date'] > pd.to_datetime('2022-01-01')].drop(columns=['15d_Return', '30d_Return', '60d_Return','Date', 'Stock']))

In [ ]:
output30 = dow30.copy()
output30 = output30[['30d_Return','Date']]

In [ ]:
output30 = output30.loc[output30['Date'] > pd.to_datetime('2022-01-01')]
output30['Predicted 30d_Return'] = preds30

In [ ]:
output30['Date'] = output30.index.map(indices_to_dates)
output30['Stock'] = output30.index.map(indices_to_stocks)

In [ ]:
output30['Date'] = pd.to_datetime(output30['Date'])
JanFeb_results30 = output30.loc[output30['Date'] > pd.to_datetime('2022-01-01')]


In [ ]:
JanFeb_results30 = JanFeb_results30.sort_values(by=['Predicted 30d_Return', 'Date'], ascending=True)

In [ ]:
JanFeb_results30.loc[JanFeb_results30['Date'] > pd.to_datetime('2023-02-14')]

In [ ]:
# January/February results from train df (model excluded Jan) achieved 0.0234 mae (+/- 2.3% return on avg)

## Profit Analysis in realistic use-case scenario

Model will be assumed to be executed every 2 weeks, where the top n predicted returns on the day of model execution will be purchased. Then, two weeks later, the positions will be exited, and positions for the n highest predicted returns on that day will be entered. Profit will be averaged by switching the day of model execution (Every other monday->Every other tuesday..etc). An overall avg profit will then be developed using this specific execution strategy. Further optimization work or even a future ML model can tackle the optimal number of n stocks to divide the allocation amongst as well as any staggerred position entry/exit.

In [ ]:
profit_analysis30 = JanFeb_results30.sort_values('Date').dropna()

In [ ]:
profit_analysis30

In [ ]:
# divide profit_analysis15 into results for each day of the business week
pa30mon = profit_analysis30.loc[profit_analysis30['Date'].dt.dayofweek == 0]
pa30tues = profit_analysis30.loc[profit_analysis30['Date'].dt.dayofweek == 1]
pa30wed = profit_analysis30.loc[profit_analysis30['Date'].dt.dayofweek == 2]
pa30th = profit_analysis30.loc[profit_analysis30['Date'].dt.dayofweek == 3]
pa30fri = profit_analysis30.loc[profit_analysis30['Date'].dt.dayofweek == 4]

pa30mon = pa30mon.sort_values(by=['Date', 'Predicted 30d_Return'], ascending=True)
pa30tues = pa30tues.sort_values(by=['Date', 'Predicted 30d_Return'], ascending=True)
pa30wed = pa30wed.sort_values(by=['Date', 'Predicted 30d_Return'], ascending=True)
pa30th = pa30th.sort_values(by=['Date', 'Predicted 30d_Return'], ascending=True)
pa30fri = pa30fri.sort_values(by=['Date', 'Predicted 30d_Return'], ascending=True)

In [ ]:
profits30 = pd.DataFrame(columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], index=range(1,10))

In [ ]:
# loop over number of stocks to split positions in each entry, and see which yields best profit
for i in range(1, 10):
    mon = pd.DataFrame(pa30mon.groupby('Date')['Predicted 30d_Return'].nlargest(i)).reset_index().set_index('level_1')
    mon['Stock'] = mon.index.map(indices_to_stocks)
    mon['30d_Return'] = mon.index.map(act_returns_mapping30)
    weeks_mon = mon.groupby('Date')['30d_Return'].mean()
    profits30['Monday'].loc[i] = weeks_mon.mean()
    
    tues = pd.DataFrame(pa30tues.groupby('Date')['Predicted 30d_Return'].nlargest(i)).reset_index().set_index('level_1')
    tues['Stock'] = tues.index.map(indices_to_stocks)
    tues['30d_Return'] = tues.index.map(act_returns_mapping30)
    weeks_tues = tues.groupby('Date')['30d_Return'].mean()
    profits30['Tuesday'].loc[i] = weeks_tues.mean()

    wed = pd.DataFrame(pa30wed.groupby('Date')['Predicted 30d_Return'].nlargest(i)).reset_index().set_index('level_1')
    wed['Stock'] = wed.index.map(indices_to_stocks)
    wed['30d_Return'] = wed.index.map(act_returns_mapping30)
    weeks_wed = wed.groupby('Date')['30d_Return'].mean()
    profits30['Wednesday'].loc[i] = weeks_wed.mean()

    th = pd.DataFrame(pa30th.groupby('Date')['Predicted 30d_Return'].nlargest(i)).reset_index().set_index('level_1')
    th['Stock'] = th.index.map(indices_to_stocks)
    th['30d_Return'] = th.index.map(act_returns_mapping30)
    weeks_th = th.groupby('Date')['30d_Return'].mean()
    profits30['Thursday'].loc[i] = weeks_th.mean()

    fri = pd.DataFrame(pa30fri.groupby('Date')['Predicted 30d_Return'].nlargest(i)).reset_index().set_index('level_1')
    fri['Stock'] = fri.index.map(indices_to_stocks)
    fri['30d_Return'] = fri.index.map(act_returns_mapping30)
    weeks_fri = fri.groupby('Date')['30d_Return'].mean()
    profits30['Friday'].loc[i] = weeks_fri.mean()

In [ ]:
profits30 # shows mean return for entering positions in stocks with the n highest predicted returns on the specified weekday
# over a 15d holding period

In [ ]:
# maximum profit: 1 stock on every sixth tuesday
tues = pd.DataFrame(pa30tues.groupby('Date')['Predicted 30d_Return'].nlargest(1)).reset_index()
tues['Stock'] = tues['level_1'].map(indices_to_stocks)
tues['30d_Return'] = tues['level_1'].map(act_returns_mapping30)
tues1 = tues.loc[(tues.index % 6 == 0)] # Takes the two stocks every third tuesday

In [ ]:
tues1

In [ ]:
mae = tues1.groupby('Date')['30d_Return'].mean()
funds = 1
for month in mae:
    funds *= (1+month)
funds

In [ ]:
funds = 0
allocation = 1.0/6.0
all1, all2, all3, all4, all5, all6 = allocation, allocation, allocation, allocation, allocation, allocation
for num, ret in enumerate(tues['15d_Return']):
    n = num % 6
    if n == 0:
        all1 *= (1+ret)
    elif n == 1:
        all2 *= (1+ret)
    elif n == 2:
        all3 *= (1+ret)
    elif n == 3:
        all4 *= (1+ret)
    elif n == 4:
        all5 *= (1+ret)
    else:
        all6 *= (1+ret)

funds = all1 + all2 + all3 + all4 + all5 + all6
funds

In [ ]:
importances30 = pd.DataFrame(reg30.get_booster().get_score(importance_type='weight'), index=[0]).transpose()
importances30.columns = ['weight']

In [ ]:
importances30.sort_values(by=['weight'], ascending = False)

## 60 Day Return Prediction

In [ ]:
reg60 = XGBRegressor()

In [ ]:
reg60.fit(x_trn, y_trn60)

In [ ]:
preds60 = reg60.predict(x_tst)

In [ ]:
print(r2_score(preds60,y_tst60))

In [ ]:
print(mean_absolute_percentage_error(preds60, y_tst60))

In [ ]:
xgb.plot_importance(reg60)